In [1]:
import sys
import sklearn
import numpy as np
import zipfile
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# np.set_printoptions(threshold=sys.maxsize)

In [2]:
train=pd.read_csv('../input/dataset/TrainingData.csv')
test=pd.read_csv('../input/dataset/TestData.csv')
sample=pd.read_csv('../input/dataset/SubmissionFormat.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def mean_iou(y_true, y_pred):
   score, up_opt = tf.metrics.mean_iou(y_true, y_pred, NUM_CLASSES)
   K.get_session().run(tf.local_variables_initializer())
   with tf.control_dependencies([up_opt]):
       score = tf.identity(score)
   return score

In [4]:
training_data = train[['Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total','FTE']]

labels = (train[['Function','Object_Type','Operating_Status','Position_Type','Pre_K', 'Reporting',
                'Sharing','Student_Type', 'Use']])

test_data = test[['Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total','FTE']]

In [5]:
training_data=training_data.drop(['Total'],axis=1)
training_data=training_data.drop(['FTE'],axis=1)

In [6]:
test_data=test_data.drop(['Total'], axis = 1)
test_data=test_data.drop(['FTE'], axis = 1)

In [7]:
for i in range(training_data.shape[1]):
    labelencoder_X=LabelEncoder();
    training_data.iloc[:,i] = training_data.iloc[:,i].astype(str)
    training_data.iloc[:,i] = training_data.iloc[:,i].str.replace('[^\w\s]','')
    training_data.iloc[:,i] = labelencoder_X.fit_transform(training_data.iloc[:,i])

In [8]:
for i in range(labels.shape[1]):
    labelencoder_X=LabelEncoder();
    labels.iloc[:,i] = labels.iloc[:,i].astype(str)
    labels.iloc[:,i] = labels.iloc[:,i].str.replace('[^\w\s]','')
    labels.iloc[:,i] = labelencoder_X.fit_transform(labels.iloc[:,i])

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
for i in range(test_data.shape[1]):
    labelencoder_X=LabelEncoder();
    test_data.iloc[:,i] = test_data.iloc[:,i].astype(str)
    test_data.iloc[:,i] = test_data.iloc[:,i].str.replace('[^\w\s]','')
    test_data.iloc[:,i] = labelencoder_X.fit_transform(test_data.iloc[:,i])

In [17]:
import keras
from keras.models import Sequential#to initialise our NN
from keras.layers import Dense#to build layers of our ANN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

x=training_data.iloc[:,0:14].values
y=labels.iloc[:,0].values
p=np.unique(y)
y=np.reshape(y,(400277,1))
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
test_data=sc.fit_transform(test_data)
np.unique(y)
model = RandomForestClassifier(n_estimators=200,random_state=0)
model.fit(X_train, y_train)
prob=np.array(model.predict_proba(test_data),dtype=object)
# print("Accuracy:",metrics.accuracy_score(y_test, prob))
for i in range(1,9):
    x=training_data.iloc[:,0:14].values
    y=labels.iloc[:,i].values
    p=np.unique(y)
    y=np.reshape(y,(400277,1))
    X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)
    sc=StandardScaler()
    X_train=sc.fit_transform(X_train)
    X_test=sc.transform(X_test)
    model = RandomForestClassifier(n_estimators=100, random_state=0)  
    model.fit(X_train, y_train)
    y_pred= np.array(model.predict_proba(test_data))
    prob=np.concatenate((prob,y_pred),axis=1)
    #     print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64

In [22]:
np.shape(prob)

(50064, 104)

In [19]:
np.savetxt("prob_RF.csv",prob, delimiter=",")